미리 만들어둔 document - term matrix 를 이용하여 LSI 를 학습합니다. 이를 위해서 SVD 를 직접 이용합니다.

In [1]:
import config
from lovit_textmining_dataset.navernews_10days import get_bow

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')

soynlp=0.0.49
added lovit_textmining_dataset


문서마다 단어의 개수가 다르기 때문에 L2 normalization 을 수행합니다.

In [2]:
from sklearn.preprocessing import normalize

x_ = normalize(x)

TruncatedSVD 를 이용하면 n_components 차원으로 문서와 단어의 공간을 바꿀 수 있습니다.

In [3]:
%%time

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100)
y = svd.fit_transform(x_)

CPU times: user 12.5 s, sys: 27.1 s, total: 39.5 s
Wall time: 16.2 s


document - term matrix 는 (30091, 9774) 의 행렬이었습니다. 9,774 개의 단어로 이뤄진 문서의 공간이 100 차원의 공간으로 바뀌었습니다.

In [4]:
print(x_.shape, y.shape)

(30091, 9774) (30091, 100)


각 단어에 대한 100 차원의 벡터는 components_ 에 저장되어 있습니다.

In [5]:
svd.components_.shape

(100, 9774)

이를 이용하여 topically similar terms 와 topically similar docs 를 찾을 수 있습니다.

Topically similar terms 를 찾는 함수를 만듭니다.

이는 다른 코드에서도 재사용 할 수 있도록 word vector (wv), vocabulary list (idx_to_vocab), vocabulary encoder (vocab_to_idx) 를 arguments 로 받을 수 있도록 만듭니다. 이 함수는 utils.py 파일에 저장해둡니다.

In [6]:
from sklearn.metrics import pairwise_distances

def most_similar_terms(term, wv, idx_to_vocab, vocab_to_idx, topn=10):
    """
    Arguments
    ---------
    term : str
        Query term
    wv : numpy.ndarray or scipy.sparse.matrix
        Word representation matrix. shape = (n_terms, dim)
    idx_to_vocab : list of str
        Index to word
    vocab_to_idx : {str:int}
        Word to index
    topn : int
        Number of most similar words
    """

    # encode term as index
    idx = vocab_to_idx.get(term, -1)
    if idx < 0:
        return []
    
    # prepare query term vector
    query_vec = wv[idx,:].reshape(1,-1)

    # compute cosine - distance
    dist = pairwise_distances(
        wv,
        query_vec,
        metric='cosine'
    ).reshape(-1)

    # find most closest terms
    # ignore query term itself
    similar_idx = dist.argsort()[1:topn+1]

    # get their distance
    similar_dist = dist[similar_idx]

    # format : [(term, similarity), ... ]
    similar_terms = [(idx, 1 - d) for idx, d in zip(similar_idx, similar_dist)]

    # decode term index to vocabulary
    similar_terms = [(idx_to_vocab[idx], d) for idx, d in similar_terms]

    # return
    return similar_terms

TruncatedSVD.components_ 는 (dim, n_terms) 의 모양입니다. transpose 를 하여 word vector 를 wv 에 저장합니다.

In [7]:
# transpose (100, 9774) -> (9774, 100)
wv = svd.components_.transpose()

'아이오아이'의 topically similar terms 입니다. 비슷한 토픽에서 등장하는 아이돌 관련 단어들이 선택됩니다. tuple 의 숫자는 cosine similarity 입니다.

In [8]:
most_similar_terms('아이오아이', wv, idx_to_vocab, vocab_to_idx)

[('신용재', 0.9614866091949101),
 ('엠카운트다운', 0.9527922506060319),
 ('너무너무너무', 0.9501852467747043),
 ('오블리스', 0.9452812378216955),
 ('빅브레인', 0.9434802906219126),
 ('세븐', 0.9253008739179873),
 ('갓세븐', 0.9214079417277032),
 ('열창', 0.8937055112927216),
 ('다비치', 0.8928198248769279),
 ('산들', 0.8835109212689727)]

'오바마'의 topically similar terms 입니다. 미국 대선이 한창이던 시절의 뉴스입니다. 미국 대선 관련 단어들이 선택됩니다. 

In [9]:
most_similar_terms('오바마', wv, idx_to_vocab, vocab_to_idx)

[('버락', 0.9693576267647734),
 ('백악관', 0.8956759825977711),
 ('꼭두각시', 0.7952453936340729),
 ('월러스', 0.7928939469966136),
 ('클린턴', 0.7918472897078195),
 ('공화당', 0.7867962954441085),
 ('주지사', 0.7859331803827629),
 ('푸틴', 0.7785625156603209),
 ('후보', 0.7774703263350555),
 ('장벽', 0.7774464118610642)]

In [10]:
most_similar_terms('트럼프', wv, idx_to_vocab, vocab_to_idx)

[('클린턴', 0.9870515872599099),
 ('승복', 0.9826816619149772),
 ('선거조작', 0.978993153693171),
 ('토론', 0.9695869708672704),
 ('공화당', 0.9691020270413129),
 ('힐러리', 0.9651825418249356),
 ('지저분', 0.9627156125014236),
 ('도널드', 0.9621907909237701),
 ('유권자들', 0.9611967543131674),
 ('음담패설', 0.9554462697909373)]

단어와 문서의 100 차원의 벡터를 학습하였으니, 이를 이용하여 해당 단어와 topically relavant 한 문서들을 검색할 수 있습니다.

각 문서에 대해 most frequent terms 를 확인하기 위하여 get_bow 함수를 만듭니다. 이 역시 utils.py 에 넣어둡니다.

In [11]:
def most_similar_docs_from_term(term, wv, dv, vocab_to_idx, topn=10):
    """
    Arguments
    ---------
    term : str
        Query term
    wv : numpy.ndarray or scipy.sparse.matrix
        Word representation matrix. shape = (n_terms, dim)
    dv : numpy.ndarray or scipy.sparse.matrix
        Document representation matrix. shape = (n_docs, dim)
    vocab_to_idx : {str:int}
        Word to index
    topn : int
        Number of most similar documents
    """

    # encode term as index
    idx = vocab_to_idx.get(term, -1)
    if idx < 0:
        return []

    # prepare query term vector
    query_vec = wv[idx,:].reshape(1,-1)

    # compute distance between query term vector and document vectors
    dist = pairwise_distances(
        dv,
        query_vec,
        metric='cosine'
    ).reshape(-1)

    # find similar document indices
    similar_doc_idx = dist.argsort()[:topn]

    # return
    return similar_doc_idx

def get_bow(doc_idx, bow, idx_to_vocab, topn=10):
    """
    Arguments
    ---------
    term : str
        Query term
    bow : scipy.sparse.matrix
        Term frequency matrix. shape = (n_docs, n_terms)
    idx_to_vocab : list of str
        Index to word
    topn : int
        Number of most frequent terms
    """

    # get term frequency submatrix
    x_sub = bow[doc_idx,:]

    # get term indices and their frequencies
    terms = x_sub.nonzero()[1]
    freqs = x_sub.data

    # format : [(term, frequency), ... ]
    bow = [(term, freq) for term, freq in zip(terms, freqs)]
    
    # sort by frequency in decreasing order
    bow = sorted(bow, key=lambda x:-x[1])[:topn]

    # decode term index to vocabulary
    bow = [(idx_to_vocab[term], freq) for term, freq in bow]

    # return
    return bow

'오바마'와 관련된 문서들입니다. 2016-10 에는 미국 대선이 이뤄지던 기간입니다.

In [12]:
similar_docs = most_similar_docs_from_term('오바마', wv, y, vocab_to_idx)

for doc_idx in similar_docs:
    bow = get_bow(doc_idx, x, idx_to_vocab)
    print('doc#={} : {}'.format(doc_idx, bow))

doc#=9615 : [('오바마', 11), ('트럼프', 11), ('대통령', 10), ('미국', 5), ('초대', 4), ('토론', 4), ('대변', 3), ('지지', 3), ('케냐', 3), ('힐러리', 3)]
doc#=9471 : [('오바마', 13), ('대통령', 12), ('트럼프', 11), ('미국', 6), ('초대', 4), ('대변', 3), ('지지', 3), ('케냐', 3), ('토론', 3), ('부인', 2)]
doc#=14951 : [('트럼프', 10), ('대통령', 7), ('클린턴', 7), ('후보', 7), ('공화당', 4), ('결과', 3), ('꼭두각시', 3), ('대선', 3), ('비방', 3), ('기자', 2)]
doc#=7256 : [('트럼프', 25), ('힐러리', 17), ('대통령', 7), ('토론', 7), ('푸틴', 6), ('미국', 5), ('여자', 5), ('끔찍', 4), ('러시아', 4), ('꼭두각시', 3)]
doc#=11929 : [('대통령', 7), ('오바마', 7), ('트럼프', 7), ('선거', 6), ('주장', 6), ('조작', 5), ('대선', 4), ('이라고', 4), ('클린턴', 4), ('후보', 4)]
doc#=14219 : [('트럼프', 6), ('오바마', 4), ('토론', 4), ('대통령', 3), ('초청', 3), ('힐러리', 3), ('3차', 2), ('공화당', 2), ('민주당', 2), ('불편', 2)]
doc#=3057 : [('토론', 24), ('트럼프', 21), ('클린턴', 18), ('3차', 8), ('대통령', 8), ('후보', 8), ('오바마', 7), ('2차', 6), ('이번', 6), ('악수', 5)]
doc#=30018 : [('토론', 12), ('트럼프', 9), ('후보', 7), ('클린턴', 6), ('3차', 5), ('대선', 4), ('대통령',

In [13]:
similar_docs = most_similar_docs_from_term('아이오아이', wv, y, vocab_to_idx)

for doc_idx in similar_docs:
    bow = get_bow(doc_idx, x, idx_to_vocab)
    print('doc#={} : {}'.format(doc_idx, bow))

doc#=16490 : [('아이오아이', 7), ('엠카운트다운', 4), ('무대', 3), ('너무너무너무', 2), ('선보', 2), ('스포츠조선', 2), ('완전체', 2), ('잠깐', 2), ('활동', 2), ('20일', 1)]
doc#=9228 : [('아이오아이', 6), ('무대', 4), ('엠카운트다운', 4), ('너무너무너무', 2), ('발랄', 2), ('완전체', 2), ('이날', 2), ('20일', 1), ('갓세븐', 1), ('금지', 1)]
doc#=21674 : [('방탄소년단', 11), ('무대', 9), ('1위', 7), ('다비치', 6), ('방송', 4), ('엠카운트다운', 4), ('올랐다', 4), ('감사', 3), ('샤이니', 3), ('10위', 2)]
doc#=16489 : [('선보', 5), ('아이오아이', 3), ('엠카운트다운', 3), ('무대', 2), ('스포츠조선', 2), ('이날', 2), ('20일', 1), ('갓세븐', 1), ('검은색', 1), ('금지', 1)]
doc#=9894 : [('아이오아이', 5), ('너무너무너무', 4), ('엠카운트다운', 3), ('출연', 3), ('걸그룹', 2), ('무대', 2), ('박진영', 2), ('발랄', 2), ('이날', 2), ('잠깐', 2)]
doc#=21249 : [('무대', 5), ('너무너무너무', 3), ('아이오아이', 3), ('컴백', 3), ('잠깐', 2), ('진영', 2), ('100', 1), ('20일', 1), ('가창력', 1), ('개개인', 1)]
doc#=18240 : [('1위', 3), ('방탄소년단', 3), ('다비치', 2), ('무대', 2), ('방송', 2), ('엠카운트다운', 2), ('100', 1), ('20일', 1), ('감사', 1), ('갓세븐', 1)]
doc#=26397 : [('1위', 4), ('무대', 4), ('아이오아이'